In [1]:
import collections

Card  = collections.namedtuple('Card',['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[5, 7, 3, 8, 4, 0, 9, 6, 2, 1]

MonkeyPatching

In [3]:
#Monkey patching Ffench Deck to make it mutable and compatible iwth random.shuffle

In [4]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [5]:
FrenchDeck.__setitem__ = set_card

In [6]:
FrenchDeck.__dict__

mappingproxy({'__module__': '__main__',
              'ranks': ['2',
               '3',
               '4',
               '5',
               '6',
               '7',
               '8',
               '9',
               '10',
               'J',
               'Q',
               'K',
               'A'],
              'suits': ['spades', 'diamonds', 'clubs', 'hearts'],
              '__init__': <function __main__.FrenchDeck.__init__(self) -> None>,
              '__len__': <function __main__.FrenchDeck.__len__(self)>,
              '__getitem__': <function __main__.FrenchDeck.__getitem__(self, position)>,
              '__dict__': <attribute '__dict__' of 'FrenchDeck' objects>,
              '__weakref__': <attribute '__weakref__' of 'FrenchDeck' objects>,
              '__doc__': None,
              '__setitem__': <function __main__.set_card(deck, position, card)>})

In [7]:
deck = FrenchDeck()
shuffle(deck)
deck[:5]

[Card(rank='10', suit='spades'),
 Card(rank='3', suit='hearts'),
 Card(rank='7', suit='clubs'),
 Card(rank='4', suit='hearts'),
 Card(rank='3', suit='clubs')]

In [8]:
#Above was an example using monkeypatching to add the setitem attribue

Subclassing an ABC

ABCs in the Standard Library

![Alt text](Image%20from%20%E2%80%9CFluent%20Python,%202nd%20Edition%E2%80%9D.png)

In [1]:
import abc

class Tombola(abc.ABC):  

    @abc.abstractmethod
    def load(self, iterable):  
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  
        return tuple(items)

ABC Syntax Details

In [2]:
import random

class BingoCage(Tombola):  

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  
        self._items = []
        self.load(items)  

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  

    def pick(self):  
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  
        self.pick()

In [3]:
#Same implementation with faster methods

import random

class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)  

    def loaded(self):  
        return bool(self._balls)

    def inspect(self):  
        return tuple(self._balls)

In [5]:
a = LottoBlower('Alpha')
a.load('beta')

A virtual subclass of an ABC

In [14]:
from random import randrange


@Tombola.register  
class TomboList(list):  

    def pick(self):
        if self:  
            position = randrange(len(self))
            return self.pop(position)  
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  

    def loaded(self):
        return bool(self)  

    def inspect(self):
        return tuple(self)

Structural Typing with ABCs

In [15]:
from typing import SupportsComplex
import numpy as np

c64 = np.complex64(3+4j)

In [18]:
isinstance(c64, complex),isinstance(c64, SupportsComplex)

(False, True)

In [20]:
import numbers

isinstance(c64, numbers.Complex)
#because numpy complex inherits numbers.Complex

True

Limitations of Runtime Protocol checks

In [21]:
import sys
sys.version

'3.11.0 | packaged by conda-forge | (main, Oct 25 2022, 06:24:40) [GCC 10.4.0]'

In [22]:
c = 3+4j

In [25]:
c.__float__()

AttributeError: 'complex' object has no attribute '__float__'

Supporting a Static Protocol

Designing a Static Protocol

Extending a Protocol
